In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools, Draw
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem import Descriptors
import seaborn as sns
from tqdm.auto import tqdm
import mols2grid

In [2]:
tqdm.pandas()

Define a function to do the clustering. While the RDKit has a function to do the clustering, we still need to cacluate fingerprints and assign cluster ids. This function performs the following steps.

- Calculate fingerprints for each molecule in mol_list
- Calculate the similarity of every molecule to every othter molecule (all pairs)
- Create a distance matrix containing 1-similarity for each pairwise similarity value
- Assign a cluster id to each molecule

In [3]:
def butina_cluster(mol_list,cutoff=0.35):
    fp_list = [rdmd.GetMorganFingerprintAsBitVect(m, 3, nBits=2048) for m in mol_list]
    dists = []
    nfps = len(fp_list)
    for i in range(1,nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fp_list[i],fp_list[:i])
        dists.extend([1-x for x in sims])
    mol_clusters = Butina.ClusterData(dists,nfps,cutoff,isDistData=True)
    cluster_id_list = [0]*nfps
    for idx,cluster in enumerate(mol_clusters,1):
        for member in cluster:
            cluster_id_list[member] = idx
    return cluster_id_list

In [6]:
df = pd.read_csv("Chembl_pIC50_curated.csv")

Let's look at the first few lines of the dataframe. 

In [7]:
df.head()

,assay,std_type,SMILES,std_value,std_units,standard_relation,molecule_chembl_id,activity_type,structure_curated,substance_type_name,pIC50
0,Inhibition of A-beta-42 production by inhibiti...,IC50,CC12CCC(C1)C(C)(C)C2NS(=O)(=O)c1ccc(F)cc1,5000.0,nM,=,CHEMBL311039,active,CC12CCC(C1)C(C)(C)C2NS(=O)(=O)c1ccc(F)cc1,organic,5.301030
1,Inhibition of A-beta-42 production by inhibiti...,IC50,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1cccs1,2700.0,nM,=,CHEMBL450926,active,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1cccs1,organic,5.568636
2,Inhibition of A-beta-42 production by inhibiti...,IC50,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,1800.0,nM,=,CHEMBL310242,active,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,organic,5.744727
3,Inhibition of A-beta-42 production by inhibiti...,IC50,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,11000.0,nM,=,CHEMBL74874,inactive,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,organic,4.958607
4,Inhibition of A-beta-42 production by inhibiti...,IC50,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,10000.0,nM,=,CHEMBL75183,active,CC12CC[C@@H](C1)C(C)(C)[C@@H]2NS(=O)(=O)c1ccc(...,organic,5.000000


We can use mols2grid to get a quick look at our dataframe.

In [8]:
mols2grid.display(df)

MolGridWidget()

Add an RDKit molecule column to the dataframe.

In [9]:
df['mol'] = [Chem.MolFromSmiles(smile) for smile in df['SMILES']]

Cluster the molecules in the dataframe and assign the cluster id to a new column.

In [10]:
%time df['Cluster'] = butina_cluster(df.mol.values)

CPU times: total: 859 ms
Wall time: 980 ms


View the dataframe with the new Cluster column

In [11]:
mols2grid.display(df,subset=["img","molecule_chembl_id","Cluster"])

MolGridWidget()

Let's look at how we could select the molecule from each cluster with the lowest LogP. First we'll calculate the LogP for each molecule and put these values into a new column called "logP".

In [12]:

df["logP"] =  [Descriptors.MolLogP(mol) for mol in df['mol']]

View the dataframe with the LogP added. Note how we use the mols2grid transform function to set the number of decimal places for LogP. Try removing this and see what happens, it's not pretty.

In [13]:
mols2grid.display(df,subset=["img","molecule_chembl_id","Cluster","logP"],transform={"logP": lambda x: f"{x:.2f}"})

MolGridWidget()

Sort the dataframe, first by Cluster then by logP.

In [14]:
df.sort_values(["Cluster","logP"],inplace=True)

In [15]:
mols2grid.display(df,subset=["img","molecule_chembl_id","Cluster","logP"],transform={"logP": lambda x: f"{x:.2f}"})

MolGridWidget()

Now let's create a new dataframe containing only the molecule from each cluster with the lowest LogP. Since we have already sorted the dataframe by cluster id and LogP, we can simply use the drop_duplicates method to drop all but the first row in each cluster.

In [16]:
df_unique = df.drop_duplicates("Cluster")

In [17]:
mols2grid.display(df_unique,subset=["img","molecule_chembl_id","Cluster","logP"],transform={"logP": lambda x: f"{x:.2f}"})

MolGridWidget()